In [ ]:
#| default_exp core

# bashnb IPython magic
> Provides `psh` persistent bash magics in Jupyter and IPython

In [ ]:
#| export
import pexpect, re, os
from pathlib import Path
from IPython.core.magic import register_cell_magic
from IPython.display import display, Javascript
from IPython.paths import get_ipython_dir
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
__file__ = './00_core.ipynb'

In [ ]:
env = dict(os.environ, TERM='dumb', PS1='', PS2='')
sh = pexpect.spawn(os.environ['SHELL'], encoding='utf-8', env=env)

echo = os.urandom(8).hex()
echo_re = re.compile(fr'^{echo}\s*$', flags=re.MULTILINE)
prompt = f'BASHNB_PROMPT_{echo}>'
sh.sendline(f'export PS1="{prompt}"')
sh.sendline('set +o vi +o emacs')
sh.sendline('echo '+echo)
sh.expect(echo_re, timeout=2)
sh.expect_exact(prompt, timeout=2)

0

In [ ]:
class ShellInterpreter:
    def __init__(self, debug=False, timeout=30, shell_path=None):
        self.debug,self.timeout = debug,timeout
        if shell_path is None: shell_path = os.environ.get('SHELL', '/bin/bash')
        env = dict(os.environ, TERM='xterm')
        env = dict(os.environ, TERM='dumb')
        self.sh = pexpect.spawn(shell_path, encoding='utf-8', env=env)
        self.echo = os.urandom(8).hex()
        self.echo_re = re.compile(fr'^{self.echo}\s*$', flags=re.MULTILINE)
        self.prompt = f'BASHNB_PROMPT_{self.echo}>'
        self.sh.sendline(f'export PS1="{self.prompt}"')
        self.sh.sendline('set +o vi +o emacs')
        self.wait_echo()
        self.wait_prompt()
        self.debug = debug

    def wait_prompt(self):
        self.sh.expect_exact(self.prompt, timeout=self.timeout)
        if self.debug: print('$', self.sh.before)

    def wait_echo(self):
        echo = 'echo '+self.echo
        self.sh.sendline(echo)
        self.sh.expect(self.echo_re, timeout=self.timeout)
        return self.sh.before.replace(echo, '').replace(self.prompt, '\n').rstrip()

    def _ex(self, s):
        if self.debug: print('#', s)
        self.sh.sendline(s)
        res = self.wait_echo()
        self.wait_prompt()
        return res
        
    def __call__(self, cmd):
        output = self._ex(cmd.rstrip())
        return output.replace(cmd + '\r\n', '', 1).rstrip()

In [ ]:
sh = ShellInterpreter(False)

In [ ]:
print(sh('ls'))

00_core.ipynb	MANIFEST.in	_quarto.yml	index.ipynb	setup.py
LICENSE		README.md	bashnb		settings.ini	styles.css


In [ ]:
shell = ShellInterpreter(debug=False)
print(shell('cd ..'))
print(shell('ls | head'), '\n')
print(shell('pwd'))

cd ..
ContextKit
FastHTML-Gallery
aimagic
answerdotai
aplnb
apswutils
audio_enhancement
bashnb
billing
blog-fastai 

/Users/jhoward/Documents/GitHub


In [ ]:
#| export
class BashMagic:
    def __init__(self):
        self.o = ShellInterpreter()

    def bash(self, line, cell=None):
        if line and not cell: cell=line
        disp = True
        if cell.endswith(';'): disp,cell = False,cell[:-1]
        res = self.o(cell) or None
        if disp: print(res)

In [ ]:
#| export
def create_magic(shell=None):
    if not shell: shell = get_ipython()
    bash_magic = BashMagic()
    shell.register_magic_function(bash_magic.bash, 'line_cell', 'psh')

In [ ]:
# Only required if you don't load the extension
create_magic()

In [ ]:
#|export
def load_ipython_extension(ipython):
    "Required function for creating magic"
    create_magic(shell=ipython)

In [ ]:
%psh pwd

/Users/jhoward/Documents/GitHub/bashnb


In [ ]:
%psh cd ..

cd ..


In [ ]:
%psh pwd

/Users/jhoward/Documents/GitHub


In [ ]:
%%psh
cd -
echo done cd

cd -
echo done cd
/Users/jhoward/Documents/GitHub/bashnb

done cd


In [ ]:
#| export
def create_ipython_config():
    "Called by `bashnb_install` to install magic"
    ipython_dir = Path(get_ipython_dir())
    cf = ipython_dir/'profile_default'/'ipython_config.py'
    cf.parent.mkdir(parents=True, exist_ok=True)
    if cf.exists() and 'bashnb' in cf.read_text(): return print('bashnb already installed!')
    with cf.open(mode='a') as f: f.write("\nc.InteractiveShellApp.extensions.append('bashnb.core')\n\n")
    print(f"Jupyter config updated at {cf}")

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()